In [1]:
"""
Imports
"""
import os, glob, re, csv, time, math, random
import numpy as np
import laspy
import open3d as o3d
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
"""
Variables
"""
# Files paths 
input_las = "Assignment02 - Cloud.las"
downsampled_las = "Downsampled_cloud.las"
filtered_las = "GroundFiltered_cloud.las"
slices_dir = "Slices_las"
accumulated_dir = "Accumulated_pairs"               
dbscan_dir = "DBSCAN_filtered"                      
ransac_dir = "RANSAC_results"                       
trees_dir = "Trees"
detected_circles_csv = "detected_circles.csv"

# Down sampling voxel size
voxel_size = 0.035

# Ground filtering
distance_threshold = 0.05                           
num_iterations = 2000                               
height_above_ground = 0.15

# Slice parameters
slice_thickness = 0.5
start_height = 0.5
end_height = 3.5

# DBSCAN clustering
dbscan_eps = 0.8                                    
dbscan_min_points = 8                               
max_points_for_dbscan = 100_000 


#RANSAC circle fitting
ransac_iterations = 1000                            
ransac_residual_threshold = 0.012                   
valid_cluster_size = (15, 400)                      
valid_radius_range = (0.05, 1.5)    
ransac_min_inliers = 20                

# Tree grouping and export
min_slices_per_tree = 3                             
radius_margin_factor = 1.8                          
z_margin = 0.15                                   
min_points_per_trunk = 100                       
z_scale_for_grouping = 0.2                        
eps_xyz = 3.0 

In [3]:
"""
Read and write to file methodes
"""

def read_las_points(filepath):
    las = laspy.read(filepath)
    points = np.vstack((las.x, las.y, las.z)).T
    return las, points


def write_las_points(filepath, points, template_las, add_height=True):
    header = laspy.LasHeader(point_format=template_las.header.point_format.id,
                             version=template_las.header.version)
    las_out = laspy.LasData(header)
    las_out.x = points[:, 0]
    las_out.y = points[:, 1]
    las_out.z = points[:, 2]

    if template_las.header.parse_crs() is not None:
        las_out.header.add_crs(template_las.header.parse_crs())

    if add_height:
        las_out.add_extra_dim(laspy.ExtraBytesParams(name="Height", type=np.float32))
        las_out.Height = points[:, 2]

    las_out.write(filepath)
    return las_out

In [4]:
"""
Downsample point cloud using voxel downsampling
"""

# Read the LAS file 
las, points = read_las_points(input_las)
print("Original point count:", len(points))

# Convert to Open3D point cloud 
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

# Voxel downsampling
pcd_down = pcd.voxel_down_sample(voxel_size)
down_pts = np.asarray(pcd_down.points)
print("Downsampled point count:", len(down_pts))

reduction = (1 - len(down_pts) / len(points)) * 100
print(f"Point cloud size reduced by {reduction:.2f}%")

las_down = write_las_points(downsampled_las, down_pts, las)

Original point count: 22176864
Downsampled point count: 12269114
Point cloud size reduced by 44.68%


In [5]:
"""
Ground filtering using RANSAC plane fitting
"""
pcd_down = o3d.geometry.PointCloud()
pcd_down.points = o3d.utility.Vector3dVector(down_pts)

plane_model, inliers = pcd_down.segment_plane(
    distance_threshold=distance_threshold,
    ransac_n=3,
    num_iterations=num_iterations
)

[a, b, c, d] = plane_model
print(f"Estimated ground plane: {a:.4f}x + {b:.4f}y + {c:.4f}z + {d:.4f} = 0")
print(f"Ground inliers: {len(inliers)} / {len(down_pts)} points")

all_points = np.asarray(pcd.points)  
distances_all = a * all_points[:, 0] + b * all_points[:, 1] + c * all_points[:, 2] + d
norm_factor = math.sqrt(a**2 + b**2 + c**2)
z_local_all = distances_all / norm_factor

mask = z_local_all > height_above_ground

filtered_points = all_points[mask]
removed_count = len(all_points) - len(filtered_points)
print(f"Filtered {removed_count} ground points, kept {len(filtered_points)} above plane.")

las_filtered = write_las_points(filtered_las, filtered_points, las)



Estimated ground plane: 0.0088x + -0.0213y + 0.9997z + 1.4365 = 0
Ground inliers: 704032 / 12269114 points
Filtered 2715497 ground points, kept 19461367 above plane.


In [6]:
"""
Slice selection and processing
"""
os.makedirs(slices_dir, exist_ok=True)

las, points = read_las_points(filtered_las)

distances = a * points[:, 0] + b * points[:, 1] + c * points[:, 2] + d
norm_factor = math.sqrt(a**2 + b**2 + c**2)
z_local = distances / norm_factor  

slice_levels = np.arange(start_height, end_height, slice_thickness)

saved_count = 0
for i, z_level in enumerate(slice_levels):
    lower, upper = z_level, z_level + slice_thickness
    mask = (z_local >= lower) & (z_local < upper)
    slice_pts = points[mask]  
    if len(slice_pts) == 0:
        continue

    out_path = os.path.join(slices_dir, f"slice_{i+1:02d}.laz")
    write_las_points(out_path, slice_pts, las)
    saved_count += 1

print(f"Finished slicing. {saved_count} slices saved in '{slices_dir}' folder.")


Finished slicing. 6 slices saved in 'Slices_las' folder.


In [7]:
"""
Accumulate points between pairs of slices onto 2D planes
"""
slice_files = sorted(glob.glob(os.path.join(slices_dir, "slice_*.laz")))

os.makedirs(accumulated_dir, exist_ok=True)

for i in range(len(slice_files) - 1):
    f1, f2 = slice_files[i], slice_files[i + 1]
    las1, pts1 = read_las_points(f1)
    las2, pts2 = read_las_points(f2)

    # Merge the two slices
    combined = np.vstack((pts1, pts2))
    combined[:, 2] = 0  # project to 2D plane (ignore z)
    
    # Save new accumulated file
    out_path = os.path.join(accumulated_dir, f"accumulated_{i+1:02d}.laz")
    write_las_points(out_path, combined, las1)
    print(f"Created accumulated pair: {out_path} ({len(combined)} pts)")



Created accumulated pair: Accumulated_pairs\accumulated_01.laz (475772 pts)
Created accumulated pair: Accumulated_pairs\accumulated_02.laz (460120 pts)
Created accumulated pair: Accumulated_pairs\accumulated_03.laz (548845 pts)
Created accumulated pair: Accumulated_pairs\accumulated_04.laz (742834 pts)
Created accumulated pair: Accumulated_pairs\accumulated_05.laz (1045443 pts)


In [8]:
"""
DBSCAN Clustering of accumulated slices with random downsampling
"""

os.makedirs(dbscan_dir, exist_ok=True)
accum_files = sorted(glob.glob(os.path.join(accumulated_dir, "accumulated_*.laz")))

# DBSCAN-parametre
for f in accum_files:
    las, pts = read_las_points(f)
    xy = pts[:, :2]

    if len(xy) < 20:
        continue

    if len(xy) > max_points_for_dbscan:
        idx = np.random.choice(len(xy), max_points_for_dbscan, replace=False)
        xy = xy[idx]

    print(f"\nClustering {os.path.basename(f)} ({len(xy)} points)")

    pcd = o3d.geometry.PointCloud()
    xy_3d = np.hstack((xy, np.zeros((len(xy), 1))))
    pcd.points = o3d.utility.Vector3dVector(xy_3d)

    labels = np.array(pcd.cluster_dbscan(eps=dbscan_eps, min_points=dbscan_min_points, print_progress=False))
    num_clusters = len(set(labels)) - (1 if -1 in labels else 0)

    print(f" → Found {num_clusters} clusters (+ {np.sum(labels==-1)} noise points)")

    # Make colors for all points
    max_label = labels.max()
    colors = plt.get_cmap("tab20")(labels / (max_label + 1 if max_label >= 0 else 1))
    colors[labels < 0] = [0, 0, 0, 1]  # black for noise

    # Plot
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    axs[0].scatter(xy[:, 0], xy[:, 1], s=1, color="gray")
    axs[0].set_title("Before DBSCAN")
    axs[0].axis("equal")

    axs[1].scatter(xy[:, 0], xy[:, 1], s=2, color=colors)
    axs[1].set_title(f"After DBSCAN ({num_clusters} clusters)")
    axs[1].axis("equal")

    plt.suptitle(os.path.basename(f))
    plt.tight_layout()
    plt.savefig(os.path.join(dbscan_dir, os.path.basename(f).replace(".laz", "_dbscan_vis.png")))
    plt.close()

    # Remove noise points and save filtered slice
    mask = labels != -1
    filtered_pts = xy[mask]
    filtered_3d = np.hstack((filtered_pts, np.zeros((len(filtered_pts), 1))))
    out_path = os.path.join(dbscan_dir, os.path.basename(f).replace(".laz", "_dbscan.laz"))
    write_las_points(out_path, filtered_3d, las)
    print(f"Saved DBSCAN-filtered slice: {out_path}")



Clustering accumulated_01.laz (100000 points)
 → Found 203 clusters (+ 265 noise points)
Saved DBSCAN-filtered slice: DBSCAN_filtered\accumulated_01_dbscan.laz

Clustering accumulated_02.laz (100000 points)
 → Found 206 clusters (+ 253 noise points)
Saved DBSCAN-filtered slice: DBSCAN_filtered\accumulated_02_dbscan.laz

Clustering accumulated_03.laz (100000 points)
 → Found 212 clusters (+ 330 noise points)
Saved DBSCAN-filtered slice: DBSCAN_filtered\accumulated_03_dbscan.laz

Clustering accumulated_04.laz (100000 points)
 → Found 218 clusters (+ 455 noise points)
Saved DBSCAN-filtered slice: DBSCAN_filtered\accumulated_04_dbscan.laz

Clustering accumulated_05.laz (100000 points)
 → Found 228 clusters (+ 472 noise points)
Saved DBSCAN-filtered slice: DBSCAN_filtered\accumulated_05_dbscan.laz


In [9]:
"""
RANSAC Circle Fitting on DBSCAN-filtered slices
"""



def fit_circle(p1, p2, p3):
    #Calculate circle center and radius from three points.
    temp = p2[0]**2 + p2[1]**2
    bc = (p1[0]**2 + p1[1]**2 - temp) / 2
    cd = (temp - p3[0]**2 - p3[1]**2) / 2
    det = (p1[0] - p2[0]) * (p2[1] - p3[1]) - (p2[0] - p3[0]) * (p1[1] - p2[1])
    if abs(det) < 1e-6:
        return None
    cx = (bc * (p2[1] - p3[1]) - cd * (p1[1] - p2[1])) / det
    cy = ((p1[0] - p2[0]) * cd - (p2[0] - p3[0]) * bc) / det
    r = math.sqrt((cx - p1[0])**2 + (cy - p1[1])**2)
    return cx, cy, r


def ransac_circle(points, iterations, threshold):
    #RANSAC for circle fitting
    best_inliers, best_circle = [], None
    n = len(points)
    if n < 3:
        return None, []
    for _ in range(iterations):
        p1, p2, p3 = points[np.random.choice(n, 3, replace=False)]
        circle = fit_circle(p1, p2, p3)
        if circle is None:
            continue
        cx, cy, r = circle
        dists = np.sqrt((points[:, 0] - cx)**2 + (points[:, 1] - cy)**2)
        inliers = np.where(np.abs(dists - r) < threshold)[0]
        if len(inliers) > len(best_inliers):
            best_inliers, best_circle = inliers, (cx, cy, r)
    return best_circle, best_inliers


def run_dbscan(xy, eps=0.18, min_points=19):
    #Run DBSCAN clustering on 2D points.
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np.column_stack((xy, np.zeros(len(xy)))))
    return np.array(pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=False))


def plot_clusters(ax, xy, title):
    ax.scatter(xy[:, 0], xy[:, 1], s=1, color="lightgray")
    ax.set_title(title)
    ax.axis("equal")


# Main RANSAC processing

os.makedirs(ransac_dir, exist_ok=True)
filtered_files = sorted(glob.glob(os.path.join(dbscan_dir, "accumulated_*_dbscan.laz")))

detected_circles = []

for f in filtered_files:
    las, pts = read_las_points(f)
    xy = pts[:, :2]
    if len(xy) < 20:
        continue

    print(f"\nProcessing {os.path.basename(f)} ...")

    # DBSCAN for separating clusters
    labels = run_dbscan(xy, eps=0.18, min_points=19)
    num_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    print(f" → Found {num_clusters} clusters")

    fig, ax = plt.subplots(figsize=(6, 6))
    plot_clusters(ax, xy, f"{os.path.basename(f)} — {num_clusters} clusters")

    total_circles = 0

    # RANSAC per cluster
    for c in range(num_clusters):
        cluster_pts = xy[labels == c]
        n_points = len(cluster_pts)
        if n_points < valid_cluster_size[0] or n_points > valid_cluster_size[1]:
            continue

        circle, inliers = ransac_circle(cluster_pts, ransac_iterations, ransac_residual_threshold)
        if (circle is None
            or len(inliers) < ransac_min_inliers):
            continue

        cx, cy, r = circle
        if not (valid_radius_range[0] <= r <= valid_radius_range[1]):
            print(f"Rejected circle (r={r:.2f} m) – outside valid range")
            continue

        detected_circles.append([os.path.basename(f), c, cx, cy, r, len(inliers)])
        total_circles += 1

        ax.scatter(cluster_pts[inliers, 0], cluster_pts[inliers, 1], s=3, color="red")
        circ = plt.Circle((cx, cy), r, color="blue", fill=False, lw=1)
        ax.add_patch(circ)
        ax.text(cx, cy, f"{r:.2f}", fontsize=6, color="blue")

    ax.set_title(f"{os.path.basename(f)} — {total_circles} fitted circles")
    plt.tight_layout()
    plt.savefig(os.path.join(ransac_dir, os.path.basename(f).replace(".laz", "_circles.png")))
    plt.close()

    print(f" → Detected {total_circles} valid circles in this layer.")

# Save results
csv_path = os.path.join(ransac_dir, detected_circles_csv)
with open(csv_path, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["File", "ClusterID", "CenterX", "CenterY", "Radius_m", "Inliers"])
    writer.writerows(detected_circles)

print(f"\n Done! Saved all results to '{ransac_dir}' and '{csv_path}'")



Processing accumulated_01_dbscan.laz ...
 → Found 231 clusters
Rejected circle (r=17.97 m) – outside valid range
Rejected circle (r=30.51 m) – outside valid range
Rejected circle (r=0.03 m) – outside valid range
Rejected circle (r=11.49 m) – outside valid range
Rejected circle (r=7.93 m) – outside valid range
 → Detected 81 valid circles in this layer.

Processing accumulated_02_dbscan.laz ...
 → Found 266 clusters
Rejected circle (r=6.53 m) – outside valid range
Rejected circle (r=0.03 m) – outside valid range
Rejected circle (r=4.45 m) – outside valid range
Rejected circle (r=42.42 m) – outside valid range
Rejected circle (r=2.00 m) – outside valid range
Rejected circle (r=4.73 m) – outside valid range
 → Detected 86 valid circles in this layer.

Processing accumulated_03_dbscan.laz ...
 → Found 306 clusters
Rejected circle (r=2.02 m) – outside valid range
Rejected circle (r=23.77 m) – outside valid range
Rejected circle (r=4.38 m) – outside valid range
Rejected circle (r=0.03 m) – 

In [10]:
"""
Where there is a best fitted circle, there should be a tree.
Load detected circles (from RANSAC) and prepare for tree grouping
"""
"""
csv_path = os.path.join(ransac_dir, detected_circles_csv)
filtered_cloud_path = filtered_las
os.makedirs(trees_dir, exist_ok=True)

# krav for at en klynge skal være et tre
min_slices_per_tree = 3

# syl/BB-parametre
radius_margin_factor = 1.8   # utvid radius litt for å få med alt
z_margin = 0.15               # legg til litt høyde-margin (m)

# klynging av (x,y,z_slice) – bruk Open3D DBSCAN på 3D-koordinater
eps_xyz = 3     # meter (tuning: 0.25–0.50 vanligvis bra)
min_points_dbscan = min_slices_per_tree  # må dukke opp i minst N slices

# --- les detected circles (ett rad per slice/cluster) ---
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"Fant ikke {csv_path}. Kjør RANSAC-delen som lager CSV først.")

rows = []
with open(csv_path, newline="") as f:
    reader = csv.DictReader(f)
    for r in reader:
        rows.append(r)

if not rows:
    raise RuntimeError("Ingen sirkler i CSV. Sjekk RANSAC-resultatene.")

# --- rekonstruer z-høyder for hver 'accumulated_XX' ---
# vi brukte 'accumulated_{i:02d}' der i = 1..(n-1), hvert par dekker to slices
# beregn base z_min fra filtered_las (slik slicing ble gjort)
_, filt_pts_all = read_las_points(filtered_cloud_path)
zmin_filtered = float(np.min(filt_pts_all[:, 2]))

# slice_levels ble tidligere: np.arange(z_min + start_height, z_min + end_height, slice_thickness)
slice_levels = np.arange(zmin_filtered + start_height, zmin_filtered + end_height, slice_thickness)

def pair_index_from_filename(basename):
    # forventer "accumulated_XX_dbscan.laz" i CSV'en (File-kolonnen)
    m = re.search(r"accumulated_(\d+)", basename)
    if not m:
        return None
    return int(m.group(1))  # 1-basert index

def z_bounds_for_pair(idx_1based):
    # pair i dekker slice i og i+1
    i = idx_1based - 1  # 0-basert
    if i < 0 or i+1 >= len(slice_levels):
        # fallback: bruk min/max vi har
        low = slice_levels.max()
        up = low + 2*slice_thickness
        return float(low), float(up)
    low = slice_levels[i]
    up  = slice_levels[i] + 2*slice_thickness
    return float(low), float(up)

# bygg punktliste for klynging og metadata
centers_xy = []
centers_xyz_for_cluster = []  # (x, y, z_center_scaled)
circle_meta = []              # per rad: dict med alt vi trenger

z_scale = 0.2  # kan senkes hvis du vil at Z skal telle mindre i DBSCAN (f.eks. 0.5)

for r in rows:
    fname = r["File"]                   # eks "accumulated_03_dbscan.laz"
    idx = pair_index_from_filename(fname)
    if idx is None: 
        continue

    cx = float(r["CenterX"]); cy = float(r["CenterY"]); rad = float(r["Radius_m"])
    z_low, z_up = z_bounds_for_pair(idx)
    z_center = 0.5 * (z_low + z_up)

    centers_xy.append([cx, cy])
    centers_xyz_for_cluster.append([cx, cy, z_center * z_scale])
    circle_meta.append({
        "file": fname,
        "pair_index": idx,
        "center": (cx, cy),
        "radius": rad,
        "z_low": z_low,
        "z_up": z_up,
        "z_center": z_center
    })

centers_xy = np.asarray(centers_xy)
centers_xyz_for_cluster = np.asarray(centers_xyz_for_cluster)

# --- cluster "trees" i (x,y,z_slice) rommet ---
pcd_centers = o3d.geometry.PointCloud()
pcd_centers.points = o3d.utility.Vector3dVector(centers_xyz_for_cluster)
labels = np.array(pcd_centers.cluster_dbscan(eps=eps_xyz, min_points=min_points_dbscan, print_progress=False))
num_clusters = len(set(labels)) - (1 if -1 in labels else 0)
print(f"Tree grouping: {num_clusters} clusters (trees) found; noise points: {(labels==-1).sum()}")

# --- bygg 3D-BB/sylinder for hvert tre ---
trees = []
tree_id = 0
for lab in sorted(set(labels)):
    if lab < 0:
        continue
    idxs = np.where(labels == lab)[0]
    if len(idxs) < min_slices_per_tree:
        continue

    sub = [circle_meta[i] for i in idxs]

    # robust center og radius
    cx = float(np.median([s["center"][0] for s in sub]))
    cy = float(np.median([s["center"][1] for s in sub]))
    r_med = float(np.median([s["radius"] for s in sub]))

    zmin = float(min(s["z_low"] for s in sub)) - z_margin
    zmax = float(max(s["z_up"]  for s in sub)) + z_margin

    tree_id += 1
    trees.append({
        "id": tree_id,
        "center": (cx, cy),
        "radius": r_med,
        "zmin": zmin,
        "zmax": zmax,
        "n_slices": len(sub),
        "pairs": sorted([s["pair_index"] for s in sub])
    })

print(f"Built BBs for {len(trees)} trees (>= {min_slices_per_tree} slices each).")

# --- eksporter punkter innenfor hver 3D-BB fra filtered_las ---
las_filt, pts_filt = read_las_points(filtered_cloud_path)
exported = 0
summary_rows = []
min_points_per_trunk = 100

for t in trees:
    cx, cy = t["center"]
    r_exp = t["radius"] * radius_margin_factor
    zmin = t["zmin"]; zmax = t["zmax"]

    dx = pts_filt[:,0] - cx
    dy = pts_filt[:,1] - cy
    dist = np.sqrt(dx*dx + dy*dy)
    z = pts_filt[:,2]

    mask = (dist <= r_exp) & (z >= zmin) & (z <= zmax)
    trunk_pts = pts_filt[mask]

    if trunk_pts.shape[0] < min_points_per_trunk:
        print(f" → Rejected tree {t['id']} ({trunk_pts.shape[0]} pts – too few)")
        continue

    out_path = os.path.join(trees_dir, f"tree_{t['id']:03d}_trunk.laz")
    write_las_points(out_path, trunk_pts, las_filt)
    exported += 1

    summary_rows.append({
        "tree_id": t["id"],
        "center_x": cx,
        "center_y": cy,
        "radius_m": t["radius"],
        "radius_used_m": r_exp,
        "zmin": zmin,
        "zmax": zmax,
        "slices_used": t["n_slices"],
        "pairs": t["pairs"],
        "points_exported": int(trunk_pts.shape[0]),
        "file": out_path
    })

print(f"Exported {exported} tree trunks to '{trees_dir}'.")



# litt sanity info
if len(trees) < 10:
    print("Fewer than 10 trees detected. Juster eps_xyz (0.25–0.5), min_slices_per_tree, eller tidligere DBSCAN/RANSAC-parametre.")


# Quick 3D plot of detected trees
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')

for t in trees:
    cx, cy = t["center"]
    zmin, zmax = t["zmin"], t["zmax"]
    ax.plot([cx, cx], [cy, cy], [zmin, zmax], color='green')
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
plt.title(f"{len(trees)} Detected Trees")
plt.show()

""" 

<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
C:\Users\aasne\AppData\Local\Temp\ipykernel_19388\3263201343.py:5: SyntaxWarning: invalid escape sequence '\d'
  """


'\ncsv_path = os.path.join(ransac_dir, detected_circles_csv)\nfiltered_cloud_path = filtered_las\nos.makedirs(trees_dir, exist_ok=True)\n\n# krav for at en klynge skal være et tre\nmin_slices_per_tree = 3\n\n# syl/BB-parametre\nradius_margin_factor = 1.8   # utvid radius litt for å få med alt\nz_margin = 0.15               # legg til litt høyde-margin (m)\n\n# klynging av (x,y,z_slice) – bruk Open3D DBSCAN på 3D-koordinater\neps_xyz = 3     # meter (tuning: 0.25–0.50 vanligvis bra)\nmin_points_dbscan = min_slices_per_tree  # må dukke opp i minst N slices\n\n# --- les detected circles (ett rad per slice/cluster) ---\nif not os.path.exists(csv_path):\n    raise FileNotFoundError(f"Fant ikke {csv_path}. Kjør RANSAC-delen som lager CSV først.")\n\nrows = []\nwith open(csv_path, newline="") as f:\n    reader = csv.DictReader(f)\n    for r in reader:\n        rows.append(r)\n\nif not rows:\n    raise RuntimeError("Ingen sirkler i CSV. Sjekk RANSAC-resultatene.")\n\n# --- rekonstruer z-høyder

In [11]:
"""
Where there is a best fitted circle, there should be a tree.
Load detected circles (from RANSAC) and prepare for tree grouping.
"""

csv_path = os.path.join(ransac_dir, detected_circles_csv)
filtered_cloud_path = filtered_las
os.makedirs(trees_dir, exist_ok=True)

if not os.path.exists(csv_path):
    raise FileNotFoundError(f"{csv_path} not found. Run RANSAC first.")

with open(csv_path, newline="") as f:
    rows = list(csv.DictReader(f))
if not rows:
    raise RuntimeError("No detected circles found in CSV.")


In [12]:
"""
For every tree, detect circles on ≥3 slices.
Reconstruct Z positions for each circle and cluster them in (x, y, z_slice).
"""
# Compute slice levels
_, filt_pts = read_las_points(filtered_cloud_path)
zmin = np.min(filt_pts[:, 2])
slice_levels = np.arange(zmin + start_height, zmin + end_height, slice_thickness)

def get_index(fname):
    m = re.search(r"accumulated_(\d+)", fname)
    return int(m.group(1)) if m else None

def z_bounds(idx):
    i = idx - 1
    if i < 0 or i+1 >= len(slice_levels):
        low = slice_levels.max()
        return float(low), float(low + 2*slice_thickness)
    return float(slice_levels[i]), float(slice_levels[i] + 2*slice_thickness)

# Build feature list for DBSCAN
z_scale = z_scale_for_grouping
centers, meta = [], []
for r in rows:
    idx = get_index(r["File"])
    if not idx: continue
    cx, cy, rad = map(float, (r["CenterX"], r["CenterY"], r["Radius_m"]))
    z_low, z_up = z_bounds(idx)
    zc = 0.5*(z_low + z_up)
    centers.append([cx, cy, zc*z_scale])
    meta.append(dict(center=(cx,cy), radius=rad, z_low=z_low, z_up=z_up, idx=idx))

centers = np.array(centers)

# DBSCAN on (x, y, scaled z)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(centers)
labels = np.array(pcd.cluster_dbscan(eps=eps_xyz, min_points=min_slices_per_tree))
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
print(f"Found {n_clusters} tree clusters; noise: {(labels==-1).sum()}")


Found 72 tree clusters; noise: 70


In [13]:
"""
Build 3D bounding boxes (cylinders) for each detected tree.
"""
trees = []
for lab in sorted(set(labels)):
    if lab < 0: continue
    ids = np.where(labels == lab)[0]
    if len(ids) < min_slices_per_tree: continue
    sub = [meta[i] for i in ids]

    cx = np.median([s["center"][0] for s in sub])
    cy = np.median([s["center"][1] for s in sub])
    r  = np.median([s["radius"] for s in sub])
    zmin = min(s["z_low"] for s in sub) - z_margin
    zmax = max(s["z_up"] for s in sub) + z_margin

    trees.append(dict(center=(cx,cy), radius=r, zmin=zmin, zmax=zmax, n=len(sub)))
print(f"Built {len(trees)} tree bounding boxes (≥{min_slices_per_tree} slices each).")


Built 72 tree bounding boxes (≥3 slices each).


In [14]:
"""
Export 3D point clouds within each bounding box (tree trunks)
"""
las_filt, pts = read_las_points(filtered_cloud_path)
exported = 0

for i, t in enumerate(trees, 1):
    cx, cy, r = t["center"][0], t["center"][1], t["radius"]*radius_margin_factor
    zmin, zmax = t["zmin"], t["zmax"]
    dx, dy = pts[:,0]-cx, pts[:,1]-cy
    dist = np.sqrt(dx*dx + dy*dy)
    z = pts[:,2]
    mask = (dist <= r) & (z >= zmin) & (z <= zmax)
    trunk = pts[mask]

    if len(trunk) < min_points_per_trunk:
        print(f" → Rejected tree {i} ({len(trunk)} pts)")
        continue

    out = os.path.join(trees_dir, f"tree_{i:03d}_trunk.laz")
    write_las_points(out, trunk, las_filt)
    exported += 1

print(f"Exported {exported} tree trunks to '{trees_dir}'.")



 → Rejected tree 2 (0 pts)
 → Rejected tree 4 (0 pts)
 → Rejected tree 6 (0 pts)
 → Rejected tree 10 (7 pts)
 → Rejected tree 15 (0 pts)
 → Rejected tree 16 (0 pts)
 → Rejected tree 22 (0 pts)
 → Rejected tree 26 (0 pts)
 → Rejected tree 35 (0 pts)
 → Rejected tree 36 (0 pts)
 → Rejected tree 38 (0 pts)
 → Rejected tree 42 (0 pts)
 → Rejected tree 46 (0 pts)
 → Rejected tree 47 (0 pts)
 → Rejected tree 53 (0 pts)
 → Rejected tree 56 (0 pts)
 → Rejected tree 57 (0 pts)
 → Rejected tree 58 (0 pts)
 → Rejected tree 61 (0 pts)
 → Rejected tree 62 (57 pts)
 → Rejected tree 65 (0 pts)
 → Rejected tree 68 (0 pts)
 → Rejected tree 69 (0 pts)
 → Rejected tree 70 (0 pts)
 → Rejected tree 71 (31 pts)
 → Rejected tree 72 (0 pts)
Exported 46 tree trunks to 'Trees'.
